# DEEP SORT 物件追蹤

  2024/07/03  
 * numpy 1.24後移除 np.float 我真的是會謝
 * 我懶得加軌跡了這程式太舊了一堆東西要改  


## 下載套件

這邊將numpy版本下調至1.23安裝完成後系統會跳出視窗詢問是否重啟，請選擇重啟  
安裝資訊中會有顯示有套件不支援,不用理它，我們這次用不到

In [ ]:
!git clone https://github.com/nwojke/deep_sort.git
!wget https://motchallenge.net/sequenceVideos/MOT16-01-raw.mp4
!gdown --id 1bB66hP9voDXuoBoaCcKYY7a8IYzMMs4P --output deep.pb
%pip install ultralytics
%pip install numpy==1.23.5

## 程式有以下兩處要改:
1. /content/deep_sort/deep_sort/linear_assignment.py  
    第4行改為
    ```python=4
    from scipy.optimize import linear_sum_assignment as linear_assignment
    ```
    58行刪除，並貼上以下程式
    ```
    x,y = linear_assignment(cost_matrix)
    indices = np.array(list(zip(x, y)))
    ```

2. /content/deep_sort/deep_sort/track.py  
    第81行，添加
    ```
    import random
    self.color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    ```

改完記得使用ctrl + s 進行存檔

## 宣告deep sort 特徵提取函式

原生deep sort 是先將特徵提取結果存在npy檔內，再由物件追蹤算法進行讀取並追蹤
因此必須要稍微改造一下

In [1]:
# vim: expandtab:ts=4:sw=4
import os
import errno
import argparse
import numpy as np
import cv2
import tensorflow as tf


def _run_in_batches(f, data_dict, out, batch_size):
    data_len = len(out)
    num_batches = int(data_len / batch_size)

    s, e = 0, 0
    for i in range(num_batches):
        s, e = i * batch_size, (i + 1) * batch_size
        batch_data_dict = {k: v[s:e] for k, v in data_dict.items()}
        out[s:e] = f(batch_data_dict)
    if e < len(out):
        batch_data_dict = {k: v[e:] for k, v in data_dict.items()}
        out[e:] = f(batch_data_dict)


def extract_image_patch(image, bbox, patch_shape):
    """Extract image patch from bounding box.

    Parameters
    ----------
    image : ndarray
        The full image.
    bbox : array_like
        The bounding box in format (x, y, width, height).
    patch_shape : Optional[array_like]
        This parameter can be used to enforce a desired patch shape
        (height, width). First, the `bbox` is adapted to the aspect ratio
        of the patch shape, then it is clipped at the image boundaries.
        If None, the shape is computed from :arg:`bbox`.

    Returns
    -------
    ndarray | NoneType
        An image patch showing the :arg:`bbox`, optionally reshaped to
        :arg:`patch_shape`.
        Returns None if the bounding box is empty or fully outside of the image
        boundaries.

    """
    bbox = np.array(bbox)
    if patch_shape is not None:
        # correct aspect ratio to patch shape
        target_aspect = float(patch_shape[1]) / patch_shape[0]
        new_width = target_aspect * bbox[3]
        bbox[0] -= (new_width - bbox[2]) / 2
        bbox[2] = new_width

    # convert to top left, bottom right
    bbox[2:] += bbox[:2]
    bbox = bbox.astype(np.int32)

    # clip at image boundaries
    bbox[:2] = np.maximum(0, bbox[:2])
    bbox[2:] = np.minimum(np.asarray(image.shape[:2][::-1]) - 1, bbox[2:])
    if np.any(bbox[:2] >= bbox[2:]):
        return None
    sx, sy, ex, ey = bbox
    image = image[sy:ey, sx:ex]
    image = cv2.resize(image, tuple(patch_shape[::-1]))
    return image


class ImageEncoder(object):

    def __init__(self, checkpoint_filename, input_name="images",
                 output_name="features"):
        self.session = tf.compat.v1.Session()
        with tf.io.gfile.GFile(checkpoint_filename, "rb") as file_handle:
            graph_def = tf.compat.v1.GraphDef()
            graph_def.ParseFromString(file_handle.read())
        tf.import_graph_def(graph_def, name="net")
        self.input_var = tf.compat.v1.get_default_graph().get_tensor_by_name(
            "%s:0" % input_name)
        self.output_var = tf.compat.v1.get_default_graph().get_tensor_by_name(
            "%s:0" % output_name)

        assert len(self.output_var.get_shape()) == 2
        assert len(self.input_var.get_shape()) == 4
        self.feature_dim = self.output_var.get_shape().as_list()[-1]
        self.image_shape = self.input_var.get_shape().as_list()[1:]

    def __call__(self, data_x, batch_size=32):
        out = np.zeros((len(data_x), self.feature_dim), np.float64)
        _run_in_batches(
            lambda x: self.session.run(self.output_var, feed_dict=x),
            {self.input_var: data_x}, out, batch_size)
        return out


def create_box_encoder(model_filename, input_name="images",
                       output_name="features", batch_size=32):
    image_encoder = ImageEncoder(model_filename, input_name, output_name)
    image_shape = image_encoder.image_shape

    def encoder(image, boxes):
        image_patches = []
        for box in boxes:
            patch = extract_image_patch(image, box, image_shape[:2])
            if patch is None:
                print("WARNING: Failed to extract image patch: %s." % str(box))
                patch = np.random.uniform(
                    0., 255., image_shape).astype(np.uint8)
            image_patches.append(patch)
        image_patches = np.asarray(image_patches)
        return image_encoder(image_patches, batch_size)

    return encoder


def generate_detections(encoder,image,rows):
    """Generate detections with features.

    Parameters
    ----------
    encoder : Callable[image, ndarray] -> ndarray
        The encoder function takes as input a BGR color image and a matrix of
        bounding boxes in format `(x, y, w, h)` and returns a matrix of
        corresponding feature vectors.
    mot_dir : str
        Path to the MOTChallenge directory (can be either train or test).
    output_dir
        Path to the output directory. Will be created if it does not exist.
    detection_dir
        Path to custom detections. The directory structure should be the default
        MOTChallenge structure: `[sequence]/det/det.txt`. If None, uses the
        standard MOTChallenge detections.

    """
    features = encoder(image, rows[:, 0:4].copy())
    detections_out = [np.r_[(row, feature)] for row, feature in zip(rows, features)]
    return detections_out


##  宣告畫框程式

In [2]:
def draw_track_box(track,img):
    i = len(track.tracks)
    for trk in reversed(track.tracks):
        d = trk.to_tlbr()
        if (trk.time_since_update < 1) and (trk.hits >= track.n_init):
            cv2.rectangle(img, (int(d[0]), int(d[1])), (int(d[2]),int(d[3])), trk.color, 2)
            cv2.putText(img, str(trk.track_id), (int(d[0])-20, int(d[1])-20), cv2.FONT_HERSHEY_SIMPLEX,
              1, (trk.color), 1, cv2.LINE_AA)
        i -= 1
    return img
    pass

## 使用YOLOV8進行偵測，並使用偵測結果進行追蹤

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker

# Load a model
model = YOLO("yolov8n.pt")  # pretrained YOLOv8n model
cap = cv2.VideoCapture('./MOT16-01-raw.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640,480))
encoder = create_box_encoder("deep.pb")
metric = nn_matching.NearestNeighborDistanceMetric(
    "cosine", 0.2, None)
tracker = Tracker(metric)
# Run batched inference on a list of images
while True:
    ret,img = cap.read()
    if not ret:
        break
    result = model([img],conf=0.3)[0]
# Process results list
    datas = np.asarray(result.boxes.data.cpu())
    datas[:,2:4]=np.asarray(result.boxes.xywh.cpu())[:,2:4]
    datas = datas[np.where(datas[:,-1]==0)]
    features = generate_detections(encoder,img,datas)
    track_data = []
    for i,feature in enumerate(features):
        box_xywh = feature[0:4]
        box_conf = feature[4]
        track_data.append(Detection(box_xywh,datas[i][4],feature[6:]))

    tracker.predict()
    tracker.update(track_data)
    img = draw_track_box(tracker,img)
    out.write(cv2.resize(img,(640,480)))


## 補充：如果改完成程式後沒有反應
那可能是因為你已經先執行過一次了，colab 為了節省計算資源不會再載入程式因此請手動載入一次

# 超級麻煩

In [ ]:
import importlib
from deep_sort import deep_sort
importlib.reload(deep_sort.detection)
importlib.reload(deep_sort.iou_matching)
importlib.reload(deep_sort.kalman_filter)
importlib.reload(deep_sort.tracker)
importlib.reload(deep_sort.nn_matching)
importlib.reload(deep_sort.linear_assignment)
importlib.reload(deep_sort.track)